# Converting membership's catalogs to convenient fits files

The format as it is before the conversion do not allow us to use it conveniently

-------------

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import GCRCatalogs
from GCRCatalogs.helpers.tract_catalogs import tract_filter, sample_filter
from GCRCatalogs import GCRQuery
from astropy.table import Table
from astropy.io import fits
from astropy.io import ascii
import pandas as pd

### Listing variables names if needed

In [2]:
#print(cat.list_all_quantities())
#print(amico_data.colnames)

### Loading catalogs and converting them to clevar tables

In [3]:
cat_names = ['9943_map_associations', '9942_map_associations', '9816_map_associations', '9815_map_associations', '9814_map_associations', '9687_map_associations', '9686_map_associations',
             '9559_map_associations', '10450_map_associations', '10327_map_associations', '10326_map_associations', '10200_map_associations', '10199_map_associations',
             '10198_map_associations', '10072_map_associations', '10071_map_associations']

In [4]:
tile_names = []
for i in range(len(cat_names)):
    tile_names.append(cat_names[i].split("_")[0])

In [5]:
cl_inpath = '/sps/lsst/users/tguillem/DESC/desc_april_2022/cluster_challenge/clevar_catalogs/amico/map_detections_refined_noBuffer_all.fits'
amico_cl_data = Table.read(cl_inpath)['ID', 'Xphys', 'Yphys', 'Zphys','LAMBSTAR', 'SN_NO_CLUSTER', 'SN', 'uid', 'tile']

-------------

### New Functions

In [17]:
def cat_to_table_cut(table, cut, tile):
    new_table = {'mb_id' : [], 'cl_id' : [], 'prob' : [], 'uid' : [], 'ra' :[], 'dec':[],'z':[], 'lambstar' : [], 'SN' : []}
    for i in range(len(table['col2'])):
        n = table['col2'][i].count('\t')
        cat_split = table['col2'][i].split("\t", n)
        n_detec = int(cat_split[1])
        for j in range(n_detec):
            if (float(cat_split[-1-j]) >= cut) and int(cat_split[2+j]) in amico_cl_data[amico_cl_data['tile'] == tile]['ID']:              
                new_table['mb_id'].append(int(table['col1'][i]))
                new_table['cl_id'].append(int(cat_split[2+j]))
                new_table['prob'].append(float(cat_split[-1-j]))
                amico_cl = amico_cl_data[(amico_cl_data['tile'] == tile) * (amico_cl_data['ID'] == int(cat_split[2+j]))]
                new_table['uid'].append(amico_cl['uid'][0])
                new_table['ra'].append(amico_cl['Xphys'][0])
                new_table['dec'].append(amico_cl['Yphys'][0])
                new_table['z'].append(amico_cl['Zphys'][0])
                new_table['lambstar'].append(amico_cl['LAMBSTAR'][0])
                new_table['SN'].append(amico_cl['SN'][0])
    t = Table(new_table)
    return t

In [18]:
for name in tile_names:
    cat_path = ('/sps/lsst/users/tguillem/DESC/desc_april_2022/cluster_challenge/clevar_catalogs/amico/DETECTIONS/' + str(name) + '_map_associations.cat')
    cat_table = Table.read(cat_path, format='ascii')
    cat_t = cat_to_table_cut(cat_table, .5, int(name))
    cat_t.write('/sps/lsst/groups/clusters/amico_validation_project/catalogs/amico_cats/amico_map_associations/' + str(name) + '_map_associations.fits', overwrite=True)
    print(cat_t['mb_id'][:10], cat_t['cl_id'][:10]) #Checking if we're not writing the same tables
    print('--------done--------')

  mb_id   
----------
9816215003
9816183686
9816183680
9816182539
9815593555
9815591288
9815591191
9815306970
9815302332
9815302331 cl_id
-----
  967
 1076
 1076
  967
 1076
 1076
 1076
 1076
  967
  967
--------done--------
  mb_id   
----------
9753392356
9753374212
9753353361
9753057542
9752525099
9752509614
9752171281
9751955055
9751704017
9751643199 cl_id
-----
  549
 1735
  549
  549
 1421
 1421
 1421
 1433
  366
  366
--------done--------
  mb_id   
----------
9812501252
8943098349
8941196336
8940791239
8940598526
8940582093
8940577825
8940565847
8940565845
8940565836 cl_id
-----
 1350
   41
  168
  168
  168
  168
  168
  168
  168
  168
--------done--------
  mb_id   
----------
8941530367
8941205648
8941198493
8941185105
8940570902
8940310609
8940003045
8939680045
8939651732
8939449825 cl_id
-----
  447
  447
  447
  447
  447
  447
 1167
  957
  957
  580
--------done--------
  mb_id   
----------
8878952232
8878744638
8878744637
8878722345
8878722344
8878721401
8878714215
8

-------------

### Old Functions

In [ ]:
def cat_to_df(table):
    new_table = {'mb_id' : [], 'nb_halo' : [], 'halo_n' : [], 'halo_id' : [], 'prob' : []}
    for i in range(len(table['col2'])):
        n = table['col2'][i].count('\t')
        cat_split = table['col2'][i].split("\t", n)
        n_detec = int(cat_split[1])
        for j in range(n_detec):
            new_table['mb_id'].append(int(table['col1'][i]))
            new_table['nb_halo'].append(int(n_detec))
            new_table['halo_n'].append(1+j)
            new_table['halo_id'].append(int(cat_split[2+j]))
            new_table['prob'].append(float(cat_split[-1-j]))
    df = pd.DataFrame(new_table)
    return df

In [12]:
for name in cat_names:
    cat_path = ('/sps/lsst/users/tguillem/DESC/desc_april_2022/cluster_challenge/clevar_catalogs/amico/DETECTIONS/' + name + '.cat')
    cat_table = Table.read(cat_path, format='ascii')
    cat_df = cat_to_df(cat_table)
    t = Table.from_pandas(cat_df)
    t.write('/sps/lsst/groups/clusters/amico_validation_project/catalogs/amico_cats/amico_map_associations/' + name + '.fits', overwrite=True)
    print(t['mb_id'][:10], t['halo_id'][:10]) #Checking if we're not writing the same tables
    print('--------done--------')
# Print the table information
#print(cat_table.info)
# Access the table data
#print(cat_table.colnames)

#Following prints not good due to print missplay in the loop (corrected but not relaunched)"""

   mb_id   
-----------
10568921559
10568921559
10568919163
10568802631
10568790827
10568790827
10568790827
10568716929
10568631335
10568631335 halo_id
-------
    284
   1184
    284
   1184
   1623
    693
    284
    284
    284
   1184
--------done--------
   mb_id   
-----------
10506383233
10506383233
10506374102
10506345774
10506342185
10506195289
10506195289
10506193923
10506120263
10506021892 halo_id
-------
    348
    667
   1161
   1161
    667
   1047
    667
    667
    667
    348
--------done--------
  mb_id   
----------
9818758495
9818750241
9818672255
9818608591
9818541169
9818510658
9818510548
9818508584
9818505412
9818499332 halo_id
-------
    237
    237
    861
    237
    237
    237
    237
    237
    237
    237
--------done--------
  mb_id   
----------
9756418145
9756297075
9756286915
9756286915
9756286915
9756178917
9756097371
9756047739
9756012571
9756010718 halo_id
-------
    659
    823
    823
    197
    659
   1533
    644
   1081
   1093
    823
-

In [47]:
def id_uid_new(table, k):
    cl_list = []
    uid_list = []
    table['uid'] = np.arange(len(table['halo_id']))
    i = 0
    for cl in table['halo_id']:
        if cl not in cl_list and cl in amico_cl_data['ID']:
            cl_list.append(cl)
            table['uid'][table['halo_id'] == cl_list[i]] = k
            uid_list.append(k)
            i += 1
            k += 1
    return table, k

In [50]:
k=1
for name in tile_names[0]:
    print(name)
    cat_path = ('/sps/lsst/users/tguillem/DESC/desc_april_2022/cluster_challenge/clevar_catalogs/amico/DETECTIONS/' + name + '_map_associations.cat')
    cat_table = Table.read(cat_path, format='ascii')
    cat_df = cat_to_df_cut(cat_table, .5)
    table = Table.from_pandas(cat_df)
    t, k = id_uid_new(table, k)
    t.write('/sps/lsst/groups/clusters/amico_validation_project/catalogs/amico_cats/amico_map_associations_uid/' + name + '_map_associations.fits', overwrite=True)
    print(Table({'mbid' : t['mb_id'][:10], 'uid' : t['uid'][:10], 'haloid' : t['halo_id'][:10]}),k) #Checking if we're not writing the same tables
    print('--------done--------')
# Print the table information
#print(cat_table.info)
# Access the table data
#print(cat_table.colnames)

#Following prints not good due to print missplay in the loop (corrected but not relaunched)

9


FileNotFoundError: [Errno 2] No such file or directory: '/sps/lsst/users/tguillem/DESC/desc_april_2022/cluster_challenge/clevar_catalogs/amico/DETECTIONS/9_map_associations.cat'